In [1]:
import joblib

# Load RF model
rf_path = "../test_cases/saved_models/random_forest_temperature_sensor.pkl"
rf_package = joblib.load(rf_path)

rf_model = rf_package["model"]
rf_features = rf_package["feature_names"]

print(f"✅ Loaded Random Forest Model!")
print(f"🌟 Expected Input Features: {rf_features}")
print(f"🌳 Number of Trees in RF: {rf_model.n_estimators}")
print(f"📈 Model Classes: {rf_model.classes_}")
print(f"🔍 Model was trained on {rf_model.n_features_in_} features")


✅ Loaded Random Forest Model!
🌟 Expected Input Features: ['temperatureSensor1', 'state_idle', 'state_off', 'state_run']
🌳 Number of Trees in RF: 100
📈 Model Classes: ['Normal']
🔍 Model was trained on 4 features


In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib
import os

# Paths
DATA_PATH = "../src/devices_data/temperature_sensor_0_unified.csv"
SAVE_PATH = "../test_cases/saved_models/random_forest_temperature_sensor.pkl"

# Load original data
df = pd.read_csv(DATA_PATH)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Select features
features = ['temperatureSensor1']
state_dummies = pd.get_dummies(df['state'], prefix='state')
df = pd.concat([df, state_dummies], axis=1)
all_features = features + list(state_dummies.columns)

# Inject stronger synthetic anomalies (2% of data)
np.random.seed(42)  # reproducible
anomaly_indices = np.random.choice(df.index, size=int(len(df) * 0.02), replace=False)
df['label'] = 'Normal'
df.loc[anomaly_indices, 'temperatureSensor1'] += np.random.uniform(80, 150, size=len(anomaly_indices))  # much bigger spikes
df.loc[anomaly_indices, 'label'] = 'Anomaly'

print(f"✅ Injected {len(anomaly_indices)} strong synthetic anomalies!")

# Train RF model
X = df[all_features]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Save model
rf_package = {
    "model": rf,
    "feature_names": all_features
}
os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)
joblib.dump(rf_package, SAVE_PATH)
print(f"✅ New Random Forest model saved to {SAVE_PATH}")


✅ Injected 402 strong synthetic anomalies!
✅ New Random Forest model saved to ../test_cases/saved_models/random_forest_temperature_sensor.pkl
